# Creating a Demand Curve

In [ ]:
import nbforms
import numpy as np
from datascience import *
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

We will create a few demand curves based on the class. Start by running the cell below and logging in.

In [ ]:
form = nbforms.Form()

Please run the cell below, fill out the questions, and then click Submit. The Submit button should turn green a few seconds after you click it once it registers your responses.

In [ ]:
form.ask()

Now let's analyze the classes data by pulling in everyone's responses.

In [ ]:
demand_table = (
    form.to_table()
    .relabeled("q1", "green tea kit kat")
    .relabeled("q2", "burrito")
    .relabeled("q3", "greek tickets")
    .relabeled("q4", "iphone 13")
)
demand_table

We need to turn all of this data into numeric types and preview them using histograms, one for each good. We will apply the `float` function to each column of the table; do not worry if you do not know what the code is doing.

In [ ]:
for label in demand_table.labels:
    col = demand_table.apply(lambda s: float(s[1:]), label)
    demand_table = demand_table.drop(label).with_column(label, col)
    
demand_table

In [ ]:
demand_table.hist('green tea kit kat')
demand_table.hist('burrito')
demand_table.hist('greek tickets')
demand_table.hist('iphone 13')

We will only look at the demand for kit kats. How many people would buy green tea kit kats at a given price? Let's assume that a person would be willing to buy the good at a price less than their bid price.  

In [ ]:
# This is a column of bid values for Greeen Tea Kitkats that you've all inputted. 
kit_kats = demand_table.select('green tea kit kat')
kit_kats

In [ ]:
# This cell does some python magic. You do not need to worry about what's going on. 
prices = pd.DataFrame({'price':[0.25, 0.5, 0.75, 1.00, 1.25, 1.5,1.75,2]})
KitkatsByPrice = kit_kats.group("green tea kit kat")
kkp = KitkatsByPrice.to_df()
gtk = (
    prices
    .merge(kkp, left_on='price', how='left', right_on='green tea kit kat')
    .fillna(0).drop('green tea kit kat', axis=1)
)
kit_kats_table = Table.from_df(gtk)
Q_demand = np.flip(np.cumsum(np.flip(kit_kats_table.group("price", sum).column(1))))
gtk_demand = Table().with_columns(
    'price', prices.price, 
    'quantity', Q_demand
)
gtk_demand

In [ ]:
# Let's graph our results
gtk_demand.scatter("quantity", "price")
plt.xlabel('Quantity')
plt.ylabel('Price')
plt.title('Demand for Green Tea Kitkats');

Now let's find the slope and intercept of the line of best fit. The cell below defines some functions that you'll learn about in the later portions of Data 8.

In [ ]:
std_units = lambda a: (a - np.mean(a)) / np.std(a)
corr = lambda x, y: np.mean(std_units(x) * std_units(y))
slope = lambda x, y: corr(x, y) * np.std(y) / np.std(x)
intercept = lambda x, y: np.mean(y) - slope(x, y) * np.mean(x)

In [ ]:
slope(gtk_demand["quantity"], gtk_demand["price"])

In [ ]:
intercept(gtk_demand["quantity"], gtk_demand["price"])